# Coefficients & Residuals

In [57]:
import os
import typing as tp

In [58]:
import numpy as np
import pandas as pd
from numpy.typing import NDArray

In [59]:
UInt8s: tp.TypeAlias = NDArray[np.uint8]
Int64s: tp.TypeAlias = NDArray[np.int64]
Floats: tp.TypeAlias = NDArray[np.float64]

## 0. Meta

In [60]:
MIN: int = -5000
MAX: int = 9999
BPL: int = 12

In [61]:
here: str = os.path.abspath('')
csv_dir: str = os.path.join(here, 'build')
path_lu: str = os.path.join(csv_dir, 'lunar.csv')
path_so: str = os.path.join(csv_dir, 'solar.csv')

In [62]:
def int_frac(x: float) -> tuple[int, float]:
    '''
    Splits a float to integral and fractional parts.

    Args:
        x (float): input floating-point value
    Returns:
        int: the maximal integer `i <= x`
        float: `x - i` in [0, 1)
    '''

    xi, xf = divmod(x, 1)
    return int(xi), xf

## 1. Lunar

### 1.0. Data

In [63]:
df_lu: pd.DataFrame = pd.read_csv(path_lu)
va_lu: pd.Series = df_lu['nian'].between(MIN, MAX)
cyue: pd.Series = df_lu['cyue']
df_lu['uday'] = (df_lu['usec'] + 28800) // 86400
uday: pd.Series = df_lu['uday']
df_lu.head()

,cyue,nian,ryue,usec,uday
0,-86209,-5001,22,-219954231530,-2545767
1,-86208,-5001,24,-219951691034,-2545737
2,-86207,-5000,2,-219949157263,-2545708
3,-86206,-5000,4,-219946626852,-2545679
4,-86205,-5000,6,-219944096614,-2545649


### 1.1. N2G: (`nian`, `ryue`) to `cyue` to uday

#### 1.1.1. `nian` to `run`

In [64]:
ryue_nr: pd.Series = df_lu['ryue']
rn_nr: pd.Series = (df_lu['ryue'] & 1).astype(np.bool)
nian_nr: pd.Series = df_lu['nian'][rn_nr]
ry_nr: pd.Series = ryue_nr[rn_nr] // 2
ry_dc: dict[int, int] = dict(zip(nian_nr, ry_nr))
print(f'{len(ry_dc)} runs')

5708 runs


In [65]:
BIT_NR, IPB_NR = 4, 2
IPL_NR: int = IPB_NR * BPL
cnt_nr_va: int = (MAX - MIN + 1) + (MIN - MAX - 1) % IPL_NR
ry_nr_bt: list[int] = []
for i in range(0, cnt_nr_va, IPB_NR):
    bt: int = 0
    for j in range(IPB_NR):
        n: int = MIN + i + j
        bt |= ry_dc.get(n, 13) << (j * BIT_NR)
    ry_nr_bt.append(bt)
ry_nr_np: UInt8s = np.array(ry_nr_bt, dtype=np.uint8)
ry_nr_np = ry_nr_np.reshape((-1, BPL))
ry_nr_ln: list[list[int]] = ry_nr_np.tolist()
print(f'{len(ry_nr_bt)} bytes')

7500 bytes


#### 1.1.2. `nian` to `cyue`: linear (1-deg) regression

In [66]:
va_ny: pd.Series = df_lu['ryue'] == 2
nian_ny: pd.Series = df_lu['nian'][va_ny]
expl_ny: pd.Series = nian_ny - MIN
dp01_ny: pd.Series = df_lu['cyue'][va_ny]
coef_ny: Floats = np.polyfit(x=expl_ny, y=dp01_ny, deg=1)
c1_ny, c0_ny = tp.cast(list[float], coef_ny.tolist())
print(f'cyue = {c1_ny} * (nian - {MIN}) + {c0_ny}')

cyue = 12.368273905932467 * (nian - -5000) + -86206.92050499133


In [67]:
c1i_ny, c1f_ny = int_frac(c1_ny)
c0i_ny, c0f_ny = int_frac(c0_ny)
print(c1i_ny, c1f_ny, c0i_ny, c0f_ny)

12 0.3682739059324671 -86207 0.07949500867107417


In [ ]:
BIT_NY, IPB_NY = 1, 8
for bits_ny in range(32):
    c0ib_ny: int = c0i_ny
    c1bi_ny, _ = int_frac(c1f_ny * (1 << bits_ny))
    c0bi_ny, _ = int_frac(c0f_ny * (1 << bits_ny))
    pred_ny: pd.Series = c1i_ny * expl_ny + c0i_ny
    bias_ny: pd.Series = c1bi_ny * expl_ny + c0bi_ny
    pred_ny += bias_ny // (1 << bits_ny)
    res_ny: pd.Series = dp01_ny - pred_ny
    if res_ny.max() - res_ny.min() < (1 << BIT_NY):
        c0ib_ny += res_ny.min() - c1i_ny * MIN
        c0bi_ny -= c1bi_ny * MIN
        res_ny -= res_ny.min()
        print(f'cyue = {c1i_ny} * nian + {c0ib_ny}', end=' + ')
        print(f'({c1bi_ny} * nian + {c0bi_ny} >> {bits_ny})')
        print(f'resd in {range(res_ny.min(), res_ny.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = 12 * nian + -26207 + (12067 * nian + 60337604 >> 15)
resd in range(0, 2)


In [69]:
IPL_NY: int = IPB_NY * BPL
va_ny: pd.Series = nian_ny.between(MIN, MAX)
cnt_ny_va: int = va_ny.sum() + 1 + (-(va_ny.sum() + 1) % IPL_NY)
res_ny_va: pd.Series = res_ny[nian_ny >= MIN].iloc[:cnt_ny_va]
res_ny_bt: list[int] = []
for i in range(0, len(res_ny_va), IPB_NY):
    bt: int = 0
    for j, r in enumerate(res_ny_va.iloc[i:i+IPB_NY]):
        bt |= r << (j * BIT_NY)
    res_ny_bt.append(bt)
res_ny_np: UInt8s = np.array(res_ny_bt, dtype=np.uint8)
res_ny_np = res_ny_np.reshape((-1, BPL))
res_ny_ln: list[list[int]] = res_ny_np.tolist()
print(f'{len(res_ny_bt)} bytes')

1884 bytes


#### 1.1.3. `cyue` to uday: linear (1-deg) regression

In [70]:
coef_yd: Floats = np.polyfit(x=cyue, y=uday, deg=1)
c1_yd, c0_yd = tp.cast(list[float], coef_yd.tolist())
print(f'uday = {c1_yd} * cyue + {c0_yd}')

uday = 29.53058513341781 * cyue + 35.7110938177826


In [71]:
c1i_yd, c1f_yd = int_frac(c1_yd)
c0i_yd, c0f_yd = int_frac(c0_yd)
print(c1i_yd, c1f_yd, c0i_yd, c0f_yd)

29 0.5305851334178087 35 0.7110938177825972


In [72]:
BIT_YD, IPB_YD = 2, 4
for bits_yd in range(64):
    c0ib_yd: int = c0i_yd
    c1bi_yd, _ = int_frac(c1f_yd * (1 << bits_yd))
    c0bi_yd, _ = int_frac(c0f_yd * (1 << bits_yd))
    pred_yd: pd.Series = c1i_yd * cyue + c0i_yd
    bias_yd: pd.Series = c1bi_yd * cyue + c0bi_yd
    pred_yd += bias_yd // (1 << bits_yd)
    res_yd: pd.Series = uday - pred_yd
    if res_yd.max() - res_yd.min() < 1 << BIT_YD:
        c0ib_yd += res_yd.min()
        res_yd -= res_yd.min()
        print(f'uday = {c1i_yd} * cyue + {c0ib_yd}', end=' + ')
        print(f'({c1bi_yd} * cyue + {c0bi_yd} >> {bits_yd})')
        print(f'resd in {range(res_yd.min(), res_yd.max() + 1)}')
        break
else:
    raise ValueError('bad idea, residuals too wide')

uday = 29 * cyue + 34 + (278179 * cyue + 372817 >> 19)
resd in range(0, 4)


In [73]:
IPL_YD: int = IPB_YD * BPL
cnt_yd_va: int = va_lu.sum() + 1 + (-(va_lu.sum() + 1) % IPL_YD)
va_yd: pd.Series = df_lu['nian'] >= MIN
res_yd_va: pd.Series = res_yd[va_yd].iloc[:cnt_yd_va]
res_yd_bt: list[int] = []
for i in range(0, len(res_yd_va), IPB_YD):
    bt: int = 0
    for j, r in enumerate(res_yd_va.iloc[i:i+IPB_YD]):
        bt |= r << (j * BIT_YD)
    res_yd_bt.append(bt)
res_yd_np: UInt8s = np.array(res_yd_bt, dtype=np.uint8)
res_yd_np = res_yd_np.reshape((-1, BPL))
res_yd_ln: list[list[int]] = res_yd_np.tolist()
print(f'{len(res_yd_bt)} bytes')

46392 bytes


### 1.2. G2N: uday to `cyue` to (`nian`, `ryue`)

#### 1.2.1. uday to `cyue`: linear (1-deg) regression

In [74]:
coef_dy: Floats = np.polyfit(x=uday, y=cyue, deg=1)
c1_dy, c0_dy = tp.cast(list[float], coef_dy.tolist())
c0_dy += 0.5 # it is midpoint that is to be predicted
print(f'cyue = {c1_dy} * cday + {c0_dy}')

cyue = 0.03386319625845294 * cday + -0.7092917774042613


In [75]:
_, c1f_dy = int_frac(c1_dy)
c0i_dy, c0f_dy = int_frac(c0_dy)
print(c1f_dy, c0i_dy, c0f_dy)

0.03386319625845294 -1 0.2907082225957387


In [76]:
for bits_dy in range(64):
    c1bi_dy, _ = int_frac(c1f_dy * (1 << bits_dy))
    c0bi_dy, _ = int_frac(c0f_dy * (1 << bits_dy))
    bias_dy: pd.Series = c1bi_dy * uday + c0bi_dy
    pred_dy: pd.Series = c0i_dy + bias_dy // (1 << bits_dy)
    if np.all(pred_dy == cyue):
        print(f'cyue = {c0i_dy}', end=' + ')
        print(f'({c1bi_dy} * uday + {c0bi_dy} >> {bits_dy})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

cyue = -1 + (8877 * uday + 76207 >> 18)


#### 1.2.2. `cyue` to `nian`: linear (1-deg) regression

In [77]:
cyue_yn: pd.Series = df_lu['cyue'][df_lu['ryue'] == 2]
nian_yn: pd.Series = df_lu['nian'][df_lu['ryue'] == 2]
coef_yn: Floats = np.polyfit(x=cyue_yn, y=nian_yn, deg=1)
c1_yn, c0_yn = tp.cast(list[float], coef_yn.tolist())
c0_yn += 0.5 # similarly, midpoints are considered
print(f'nian = {c1_yn} * cyue + {c0_yn}')

nian = 0.0808520257214669 * cyue + 1970.5041542503538


In [78]:
_, c1f_yn = int_frac(c1_yn)
c0i_yn, c0f_yn = int_frac(c0_yn)
print(c1f_yn, c0i_yn, c0f_yn)

0.0808520257214669 1970 0.5041542503538494


In [79]:
for bits_yn in range(64):
    c1bi_yn, _ = int_frac(c1f_yn * (1 << bits_yn))
    c0bi_yn, _ = int_frac(c0f_yn * (1 << bits_yn))
    bias_yn: pd.Series = c1bi_yn * cyue_yn + c0bi_yn
    pred_yn: pd.Series = c0i_yn + bias_yn // (1 << bits_yn)
    if np.all(pred_yn == nian_yn):
        print(f'nian = {c0i_yn}', end=' + ')
        print(f'({c1bi_yn} * cyue + {c0bi_yn} >> {bits_yn})')
        break
else:
    raise ValueError('bad idea, residuals too wide')

nian = 1970 + (10597 * cyue + 66080 >> 17)
